In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import datetime
from dateutil.relativedelta import relativedelta
from skinet.utils.athena_tools import *
warnings.filterwarnings("ignore")

In [2]:
ym = '202011'                          # 전월 (몇월의 마감자료로 작업하고 있는지)
final_call_boundary = '2020/10/01'     # 최종 통화일 기준 (이 이후 통화건이 있으면 제외)
final_call_boundary_2 = '2020/01/01'     # 최종 통화일 기준 (이 이후 통화건이 있으면 제외)

In [3]:
# 6개월 전 연월일
ymd_minus_6month = datetime.date(int(ym[0:4]), int(ym[4:6]), 1) - relativedelta(months=6)
ymd_minus_6month = str(ymd_minus_6month.year)+str(ymd_minus_6month.month).zfill(2)+str(ymd_minus_6month.day).zfill(2)
ymd_minus_6month

'20200501'

In [4]:
# 6개월 전 연월일 (슬래쉬 형식)
ymd_minus_6month_slash = datetime.date(int(ym[0:4]), int(ym[4:6]), 1) - relativedelta(months=6)
ymd_minus_6month_slash = str(ymd_minus_6month_slash.year)+'/'+str(ymd_minus_6month_slash.month).zfill(2)+'/'+str(ymd_minus_6month_slash.day).zfill(2)
ymd_minus_6month_slash

'2020/05/01'

In [5]:
# 1년 전 연월일
ymd_minus_1year = datetime.date(int(ym[0:4]), int(ym[4:6]), 1) - relativedelta(years=1)
ymd_minus_1year = str(ymd_minus_1year.year)+str(ymd_minus_1year.month).zfill(2)+str(ymd_minus_1year.day).zfill(2)
ymd_minus_1year

'20191101'

## 2. 기 가입고객 테이블

In [6]:
conn = athena_connection()
gigaip = load_from_athena(conn, '''
WITH t1 AS (
SELECT DISTINCT b.ctnum AS ctnum_hash
     , e.ctnum
     , b.rqnum
     , b.valid_sdate
     , b.valid_edate
     , b.ins_type
     , b.ins_amt
     , b.ins_amt_type
     , b.ins_payment
     , b.ins_stat
     , b.ins_user4
     , c.ins_stat_nm
     , c.ins_amt_type_nm
	 , f.gad_start
     , d.ctnum_old
  FROM (SELECT DISTINCT ctnum, ins_type, ins_amt, ins_amt_type, ins_payment, ins_stat, rqnum, valid_sdate, valid_edate, ins_user4
             FROM sap.zsdt4150
            WHERE ins_type IN ('02', '03', '05', '07')) b
LEFT JOIN (SELECT DISTINCT ctnum, rqnum, ins_stat_nm, ins_amt_type_nm FROM mart.zsdr6051) c
       ON b.ctnum = c.ctnum
      AND b.rqnum = c.rqnum
LEFT JOIN (SELECT DISTINCT ctnum, ctnum_old FROM mart.zsdr6001 WHERE ctnum_old <> '') d
       ON b.ctnum = d.ctnum
LEFT JOIN (SELECT DISTINCT ctnum, ctnum_hash FROM adt_work.hash_info) e
       ON b.ctnum = e.ctnum_hash
LEFT JOIN (SELECT ctnum, gad_start FROM sap.zsdt4013 WHERE stmon = \'{}\') f
       ON b.ctnum = f.ctnum
)

, t2 AS (
SELECT * , 'case1' AS info FROM t1 WHERE ins_stat = '1' AND ins_type = '02' AND ins_amt_type >= '20' UNION ALL
SELECT * , 'case2' AS info FROM t1 WHERE ins_stat = '1' AND ins_type = '03' AND ins_amt_type >= '50' UNION ALL
SELECT * , 'case3' AS info FROM t1 WHERE ins_stat = '1' AND ins_user4 = 'TM'
)

SELECT DISTINCT *
  FROM t2
 WHERE valid_sdate <> '99991231'
'''.format(ym))

...tmp_db name = tmp_20201222_11_04_11_f90212b83a2e2e5497dc644d5e47c6d1eacd6992
Start creating table
drop table if exists default.tmp_20201222_11_04_11_f90212b83a2e2e5497dc644d5e47c6d1eacd6992
<< table not exist

        create  table         default.tmp_20201222_11_04_11_f90212b83a2e2e5497dc644d5e47c6d1eacd6992
        with (format='Parquet', parquet_compression='GZIP')
        as
        
WITH t1 AS (
SELECT DISTINCT b.ctnum AS ctnum_hash
     , e.ctnum
     , b.rqnum
     , b.valid_sdate
     , b.valid_edate
     , b.ins_type
     , b.ins_amt
     , b.ins_amt_type
     , b.ins_payment
     , b.ins_stat
     , b.ins_user4
     , c.ins_stat_nm
     , c.ins_amt_type_nm
	 , f.gad_start
     , d.ctnum_old
  FROM (SELECT DISTINCT ctnum, ins_type, ins_amt, ins_amt_type, ins_payment, ins_stat, rqnum, valid_sdate, valid_edate, ins_user4
             FROM sap.zsdt4150
            WHERE ins_type IN ('02', '03', '05', '07')) b
LEFT JOIN (SELECT DISTINCT ctnum, rqnum, ins_stat_nm, ins_amt_type_n

## 3. TM 실적 테이블 (계약번호, 회차, 최종접촉일, 최종접촉결과)

In [7]:
conn = athena_connection()
drop_table_in_athena(conn, 'default.tm_upsell_result_table')
tm = load_from_athena(conn, '''
WITH t1 AS (
SELECT DISTINCT a.custid, b.kunnr, CONCAT(dialdate, ' ; ', dialtime) AS dialdate, resultcd, teamcd
  FROM telepro.tm_list a
LEFT JOIN (SELECT DISTINCT ctnum, kunnr, ctnum_hash, kunnr_hash FROM adt_work.hash_info) b
      ON a.custid = b.ctnum
 WHERE teamcd IN ('313', '316')
)

, t2_temp AS (
SELECT DISTINCT kunnr, resultcd, dialdate
  FROM t1
WHERE resultcd IS NOT NULL
  AND resultcd <> ''
  AND resultcd <> ' '
)

, t2 AS(
SELECT a.kunnr, COUNT(a.resultcd) AS con_cnt, MAX(a.dialdate) AS final_dialdate
  FROM t2_temp a
LEFT JOIN (SELECT DISTINCT kunnr, kunnr_hash FROM adt_work.hash_info) b
       ON a.kunnr = b.kunnr
GROUP BY a.kunnr

)

SELECT a.*, b.resultcd
  FROM t2 a
LEFT JOIN (SELECT * FROM t1) b
       ON a.kunnr = b.kunnr
      AND a.final_dialdate = b.dialdate
''')
load_to_athena(conn, tm, 'default.tm_upsell_result_table')

<< Delete s3://adt-sandbox/_create/tables/default.tm_upsell_result_table
...tmp_db name = tmp_20201222_11_05_03_6d4488bb10c62f137fb665638ff9b51bb34c1cc7
Start creating table
drop table if exists default.tmp_20201222_11_05_03_6d4488bb10c62f137fb665638ff9b51bb34c1cc7
<< table not exist

        create  table         default.tmp_20201222_11_05_03_6d4488bb10c62f137fb665638ff9b51bb34c1cc7
        with (format='Parquet', parquet_compression='GZIP')
        as
        
WITH t1 AS (
SELECT DISTINCT a.custid, b.kunnr, CONCAT(dialdate, ' ; ', dialtime) AS dialdate, resultcd, teamcd
  FROM telepro.tm_list a
LEFT JOIN (SELECT DISTINCT ctnum, kunnr, ctnum_hash, kunnr_hash FROM adt_work.hash_info) b
      ON a.custid = b.ctnum
 WHERE teamcd IN ('313', '316')
)

, t2_temp AS (
SELECT DISTINCT kunnr, resultcd, dialdate
  FROM t1
WHERE resultcd IS NOT NULL
  AND resultcd <> ''
  AND resultcd <> ' '
)

, t2 AS(
SELECT a.kunnr, COUNT(a.resultcd) AS con_cnt, MAX(a.dialdate) AS final_dialdate
  FROM t2_tem

0

## TM 거부 이력 고객 

In [8]:
conn = athena_connection()
tm_refuse = load_from_athena(conn, '''
SELECT DISTINCT a.custid, b.kunnr, CONCAT(dialdate, ' ; ', dialtime) AS dialdate, resultcd
  FROM telepro.tm_list a
LEFT JOIN (SELECT DISTINCT ctnum, kunnr, ctnum_hash, kunnr_hash FROM adt_work.hash_info) b
      ON a.custid = b.ctnum
 WHERE resultcd = 'SE01'
''')

...tmp_db name = tmp_20201222_11_05_53_8598cb085908b38eb145b9f85b970a5a314660e2
Start creating table
drop table if exists default.tmp_20201222_11_05_53_8598cb085908b38eb145b9f85b970a5a314660e2
<< table not exist

        create  table         default.tmp_20201222_11_05_53_8598cb085908b38eb145b9f85b970a5a314660e2
        with (format='Parquet', parquet_compression='GZIP')
        as
        
SELECT DISTINCT a.custid, b.kunnr, CONCAT(dialdate, ' ; ', dialtime) AS dialdate, resultcd
  FROM telepro.tm_list a
LEFT JOIN (SELECT DISTINCT ctnum, kunnr, ctnum_hash, kunnr_hash FROM adt_work.hash_info) b
      ON a.custid = b.ctnum
 WHERE resultcd = 'SE01'

...
>> Athena Query Success!!
>> Athena Query EXCUTION COMPLETE....
>> S3 FILE GENERATION SUCCESS!! (File count: 6)
Elapsed Time = 25.91
>> Start downloading table
aws s3 cp s3://adt-sandbox/dsc/tables/5a6000fd-6dbc-42f4-9eb0-bb77dbad7bef /home/skinet/tmp/tmp_20201222_11_05_53_8598cb085908b38eb145b9f85b970a5a314660e2 --recursive
Elapsed Time =

## 1차 결과테이블 추출 (1번 ~ 15번 조건 반영)

In [9]:
df = load_from_athena(conn,'''
WITH magam AS (
SELECT a.*, b.con_cnt, b.final_dialdate, b.resultcd
     , ROW_NUMBER() OVER (PARTITION BY a.kunnr ORDER BY a.conn_type ASC, a.ctnum DESC, a.aedat DESC, a.aezet DESC) AS kunnr_rownum
  FROM default.monthly_deadline_{} a
LEFT JOIN default.tm_upsell_result_table b
       ON a.kunnr = b.kunnr
LEFT JOIN default.tm_upsell_db_temp c
       ON a.ctnum = c.ctnum
)

-- 이 하단부터 조건들

, t1 AS (
SELECT '1' AS filter, *  FROM magam WHERE cstat <> '1'
)

, t2 AS (
SELECT '2' AS filter, *  FROM magam WHERE itype_l <> '1' OR itype_m <> '02'
)

, t3 AS (
SELECT '3' AS filter, *  FROM magam WHERE gros_code > '0'
)

, t4 AS (
SELECT '4' AS filter, *  FROM magam WHERE gross_type = 'Y'
)

, t5 AS (
SELECT '5' AS filter, * FROM magam WHERE adt1_yn = 'N'
)

, t6 AS (
SELECT '6' AS filter, * FROM magam WHERE tm_yn = 'N' OR tm_yn IS NULL or tm_yn = '' or tm_yn = ' '
)

, t7 AS (
SELECT '7' AS filter, * FROM magam WHERE (itype_l = '1' AND (monexp_amt < 30000 OR monexp_amt > 400000))
                                      OR (itype_l = '4' AND (monexp_amt < 30000 OR monexp_amt > 150000))
)

, t8 AS (
SELECT '8' AS filter, * FROM magam WHERE kunnr_rownum > 1
)

, total AS (
SELECT * FROM t1 UNION ALL 
SELECT * FROM t2 UNION ALL 
SELECT * FROM t3 UNION ALL 
SELECT * FROM t4 UNION ALL 
SELECT * FROM t5 UNION ALL 
SELECT * FROM t6 UNION ALL 
SELECT * FROM t7 UNION ALL 
SELECT * FROM t8
)

, final_filter AS (
SELECT ctnum, MIN(filter) AS filter FROM total GROUP BY ctnum
)

SELECT b.filter, a.* FROM magam a LEFT JOIN final_filter b ON a.ctnum = b.ctnum 
'''.format(ym))

...tmp_db name = tmp_20201222_11_06_33_01d83cdcee4c00de989ca4741c54b9de42c86891
Start creating table
drop table if exists default.tmp_20201222_11_06_33_01d83cdcee4c00de989ca4741c54b9de42c86891
<< table not exist

        create  table         default.tmp_20201222_11_06_33_01d83cdcee4c00de989ca4741c54b9de42c86891
        with (format='Parquet', parquet_compression='GZIP')
        as
        
WITH magam AS (
SELECT a.*, b.con_cnt, b.final_dialdate, b.resultcd
     , ROW_NUMBER() OVER (PARTITION BY a.kunnr ORDER BY a.conn_type ASC, a.ctnum DESC, a.aedat DESC, a.aezet DESC) AS kunnr_rownum
  FROM default.monthly_deadline_202011 a
LEFT JOIN default.tm_upsell_result_table b
       ON a.kunnr = b.kunnr
LEFT JOIN default.tm_upsell_db_temp c
       ON a.ctnum = c.ctnum
)

-- 이 하단부터 조건들

, t1 AS (
SELECT '1' AS filter, *  FROM magam WHERE cstat <> '1'
)

, t2 AS (
SELECT '2' AS filter, *  FROM magam WHERE itype_l <> '1' OR itype_m <> '02'
)

, t3 AS (
SELECT '3' AS filter, *  FROM magam WHERE gr

In [10]:
df['filter'].fillna(0).astype(int).value_counts(dropna=False).sort_index()

0     55115
1      2880
2    673540
3      4406
4        88
5      1703
6     30498
7      2365
8      2844
Name: filter, dtype: int64

In [11]:
# 불필요한 컬럼 제거 
drop_col = ['kcalss', 'conn_dept', 'conn_emp', 'cost_dept', 'cost_emp', 'intro_asp', 'itype_sdesc', 'svc_str', 'pay_type', 'depo_type', 'kunnr_rownum']
df.drop(drop_col, axis=1, inplace=True)

In [12]:
# 혹시 몰라서 카피떠놓기
df_copy = df.copy()

In [13]:
# df = df_copy.copy()

In [14]:
# 필터링하지 않을 이상전화번호 리스트 (이상한 전화번호지만 db에 남겨놓고 TM센터에서 수기로 다른 번호 찾기 시도)
no_delete = ['0', 
'00','01','02','03','010','0101','01111','010000','010226','0100000','0101111','0105256','0109152','0109364','01000000','01001638','02000000','010000000','010010010','010111111'
,'011111111','020000000','021111111','0100000000','0100000001','0100100000','0101010101','0101111111','0101111234','0101231234','0101234567','0110000000','0111110000','0111234567','0200000000'
,'0201021021','0202222222','0111111111','0211000000','0211111112','0212341234','0211112222','0222222222','0510000000','01000000000','01000000001','01000000250','01000000602','01000001111','01000009999'
,'01000010001','01000090000','01000112111','01001000000','01001000900','01001010101','01001111111','01010000000','01010001000','01011110000','01011111018','01011111110','01011111111'
,'01001001010','01011111112','01011111118','01011111234','01011111313','01011112222','01012341234','01012345678','01022222222','01100000000','01101111111','01101234567','01111111111'
,'01111112222','01111113333','01112311231','03200000000','05100000000','05100011111','05101111111','05101112222','010000000000','010111111111','011111111111','0100000000000','01000000000000'
,'010000000000000','0','1','2','3','10','101','1111','10000','10226','100000','101111','105256','109152','109364','1000000','1001638','2000000','10000000','10010010','10111111','11111111'
,'20000000','21111111','100000000','100000001','100100000','101010101','101111111','101111234','101231234','101234567','110000000','111110000','111234567','200000000','201021021','202222222'
,'111111111','211000000','211111112','212341234','211112222','222222222','510000000','1000000000','1000000001','1000000250','1000000602','1000001111','1000009999','1000010001','1000090000'
,'1000112111','1001000000','1001000900','1001010101','1001111111','1010000000','1010001000','1011110000','1011111018','1011111110','1011111111','1001001010','1011111112','1011111118','1011111234'
,'1011111313','1011112222','1012341234','1012345678','1022222222','1100000000','1101111111','1101234567','1111111111','1111112222','1111113333','1112311231','3200000000','5100000000','5100011111'
,'5101111111','5101112222','10000000000','10111111111','11111111111','100000000000','1000000000000','10000000000000']

In [15]:
# 이상휴대폰번호 판단 함수 
# 등차수열이거나(ex.01012345678) 다 똑같은 번호거나(ex.01000000000), 위의 이상번호 리스트에 포함되어 있거나.

def valid_number(x):
      nums = list(str(x)) 
      n = len(nums)
      answer=[]
      i = 3
      
      while i <= n-3:
            if int(nums[i]) - int(nums[i+1]) == int(nums[i+1]) - int(nums[i+2]):
                  answer.append(True)
            else:
                  answer.append(False)
            i += 1
      
      if (all(answer) == True) or (str(x) in (no_delete)):
            return False
      else:
            return True     

In [16]:
# 16. 휴대폰번호 전처리 및 중복 제거

df['telf2'] = df['telf2'].str.replace('\D|\s', '', regex=True)  # 숫자가 아닌 글자랑 공백 날리기
df['telf2'] = np.where(df['telf2'].str[0] != '0', "0"+df['telf2'], df['telf2'])   # 맨 앞에 0이 없으면 0 붙여주기

df['telf_valid_1'] = df['telf2'].str.contains('^01([0|1|6|7|8|9]?)-?([0-9]{3,4})-?([0-9]{4})$', regex=True)  # 자릿수가 맞는지 확인하기
df['telf_valid_2'] = df['telf2'].apply(valid_number)                                                        # 위에 정의한 함수를 통해 valid한지 확인하기
df['telf_valid_final'] = np.where((df['telf_valid_1'] == True) & (df['telf_valid_2'] == True), True, False)
df['telf_valid_final'] = df['telf_valid_final'].map({True : 1, False : 0})                                  # 최종적으로 valid한지 확인

df = df.sort_values(by=['conn_type', 'ctnum', 'aedat', 'aezet'], axis=0, ascending=[True, False, False, False])  # 정렬해서 중복된 거 제거                                 

df.loc[(df['filter'].isnull()==True) & (df[(df['filter'].isnull()==True) & ((df['telf_valid_final']==1))].duplicated(subset='telf2', keep='first'))
           , 'filter'] = 9

In [17]:
#16-2 직원 휴대폰번호 제거 
conn = athena_connection()
emp = load_from_athena(conn,
'''
SELECT DISTINCT *
  FROM sap.zsdt0070
''')

...tmp_db name = tmp_20201222_11_07_45_c7be5de2d456dcd103d0eda043e4a8500924d23d
Start creating table
drop table if exists default.tmp_20201222_11_07_45_c7be5de2d456dcd103d0eda043e4a8500924d23d
<< table not exist

        create  table         default.tmp_20201222_11_07_45_c7be5de2d456dcd103d0eda043e4a8500924d23d
        with (format='Parquet', parquet_compression='GZIP')
        as
        
SELECT DISTINCT *
  FROM sap.zsdt0070

.
>> Athena Query Success!!
>> Athena Query EXCUTION COMPLETE....
>> S3 FILE GENERATION SUCCESS!! (File count: 6)
Elapsed Time = 14.67
>> Start downloading table
aws s3 cp s3://adt-sandbox/dsc/tables/e8c45a9a-44f1-434d-82e1-920e88709704 /home/skinet/tmp/tmp_20201222_11_07_45_c7be5de2d456dcd103d0eda043e4a8500924d23d --recursive
Elapsed Time = 7.33
>> Loading data
Dropping tmp db
<< Delete s3://adt-sandbox/dsc/tables/e8c45a9a-44f1-434d-82e1-920e88709704
Removing tmp_dir!
Elapsed Time for downloading and loading = 6.03
Total Elapsed Time = 28.03


In [18]:
emp['tel_hand'] = emp['tel_hand'].str.replace('\D|\s', '', regex=True)  # 숫자가 아닌 글자랑 공백 날리기
emp['tel_hand'] = np.where(emp['tel_hand'].str[0] != '0', "0"+emp['tel_hand'], emp['tel_hand'])   # 맨 앞에 0이 없으면 0 붙여주기
df.loc[(df['filter'].isnull()==True) & (df['telf2'].isin(emp['tel_hand'])) & (~df['telf2'].isin(no_delete))  , 'filter'] = 9.5

In [19]:
# 17. 주소 중복내역 제거

df = df.sort_values(by=['conn_type', 'ctnum', 'aedat', 'aezet'], axis=0, ascending=[True, False, False, False])  # 정렬해서 중복된 거 제거                                 
df.loc[(df['filter'].isnull()==True) & (df.duplicated(subset=['gad_add01', 'gad_add02'], keep='first')), 'filter'] = 10

In [20]:
# 18. 최근 1년 이내 민원접수 고객
conn = athena_connection()
voc = load_from_athena(conn, '''
SELECT DISTINCT b.ctnum, a.receiptdate, a.receipttime
  FROM telepro.tb_voc a
LEFT JOIN (SELECT DISTINCT ctnum, kunnr, ctnum_hash, kunnr_hash FROM adt_work.hash_info) b
       ON a.s25csq = b.ctnum_hash 
 WHERE a.receiptdate >= \'{}\'
'''.format(ymd_minus_1year))

...tmp_db name = tmp_20201222_11_08_17_ccb6e8aef961c27071ea7e4a3cb13798a99b5d2d
Start creating table
drop table if exists default.tmp_20201222_11_08_17_ccb6e8aef961c27071ea7e4a3cb13798a99b5d2d
<< table not exist

        create  table         default.tmp_20201222_11_08_17_ccb6e8aef961c27071ea7e4a3cb13798a99b5d2d
        with (format='Parquet', parquet_compression='GZIP')
        as
        
SELECT DISTINCT b.ctnum, a.receiptdate, a.receipttime
  FROM telepro.tb_voc a
LEFT JOIN (SELECT DISTINCT ctnum, kunnr, ctnum_hash, kunnr_hash FROM adt_work.hash_info) b
       ON a.s25csq = b.ctnum_hash 
 WHERE a.receiptdate >= '20191101'

......
>> Athena Query Success!!
>> Athena Query EXCUTION COMPLETE....
>> S3 FILE GENERATION SUCCESS!! (File count: 6)
Elapsed Time = 46.76
>> Start downloading table
aws s3 cp s3://adt-sandbox/dsc/tables/3bb5f790-83fc-42aa-8274-fabc9405d97f /home/skinet/tmp/tmp_20201222_11_08_17_ccb6e8aef961c27071ea7e4a3cb13798a99b5d2d --recursive
Elapsed Time = 7.13
>> Loading d

In [21]:
df.loc[(df['filter'].isnull()==True) & (df['ctnum'].isin(voc['ctnum'])) , 'filter'] = 11

In [22]:
# 19. 해약 문의
conn = athena_connection()
churn = load_from_athena(conn, '''
WITH t1 AS (
SELECT DISTINCT a.s25csq, b.gubun AS cd, a.talkenddate, '이관내역 기준' AS info
  FROM telepro.tb_contact a
LEFT JOIN telepro.tb_trans_contact b
       ON a.contactid = b.contactid
 WHERE b.gubun IN ('0025', '0026')
   AND a.talkenddate >= \'{}\'
   
UNION ALL 

SELECT DISTINCT s25csq, contactmiddlecd AS cd, talkenddate, '상담유형(중) 기준' AS info
  FROM telepro.tb_contact
 WHERE contactmiddlecd IN ('J50J53', 'K50K56', 'J50J54')
   AND talkenddate >= \'{}\'
   
UNION ALL

SELECT DISTINCT s25csq, contactminorcd AS cd, talkenddate, '상담유형(소) 기준' AS info
  FROM telepro.tb_contact
 WHERE contactminorcd IN ('B50B51B83', 'A01A08A25', 'G50G52G64')
   AND talkenddate >= \'{}\'
)

SELECT b.ctnum, a.cd, a.talkenddate, a.info
  FROM t1 a
LEFT JOIN (SELECT DISTINCT ctnum, kunnr, ctnum_hash, kunnr_hash FROM adt_work.hash_info) b
       ON a.s25csq = b.ctnum_hash
'''.format(ymd_minus_6month_slash, ymd_minus_6month_slash, ymd_minus_6month_slash))

...tmp_db name = tmp_20201222_11_09_17_9828052866bcce722b2bca5dfa793507bb49430b
Start creating table
drop table if exists default.tmp_20201222_11_09_17_9828052866bcce722b2bca5dfa793507bb49430b
<< table not exist

        create  table         default.tmp_20201222_11_09_17_9828052866bcce722b2bca5dfa793507bb49430b
        with (format='Parquet', parquet_compression='GZIP')
        as
        
WITH t1 AS (
SELECT DISTINCT a.s25csq, b.gubun AS cd, a.talkenddate, '이관내역 기준' AS info
  FROM telepro.tb_contact a
LEFT JOIN telepro.tb_trans_contact b
       ON a.contactid = b.contactid
 WHERE b.gubun IN ('0025', '0026')
   AND a.talkenddate >= '2020/05/01'
   
UNION ALL 

SELECT DISTINCT s25csq, contactmiddlecd AS cd, talkenddate, '상담유형(중) 기준' AS info
  FROM telepro.tb_contact
 WHERE contactmiddlecd IN ('J50J53', 'K50K56', 'J50J54')
   AND talkenddate >= '2020/05/01'
   
UNION ALL

SELECT DISTINCT s25csq, contactminorcd AS cd, talkenddate, '상담유형(소) 기준' AS info
  FROM telepro.tb_contact
 WHERE con

In [23]:
df.loc[(df['filter'].isnull()==True) & (df['ctnum'].isin(churn['ctnum'])) , 'filter'] = 12

In [24]:
# 20. tm 거부
df.loc[(df['filter'].isnull()==True) & df['kunnr'].isin(tm_refuse['kunnr']), 'filter'] = 13

In [25]:
# 21. 두낫콜
donotcall = pd.DataFrame()
donotcall['휴대폰번호'] = df['telf2'].drop_duplicates(keep='first')
donotcall = donotcall.sort_values('휴대폰번호')
donotcall.index.name = 'index'
donotcall.to_csv('donotcall_list.csv', encoding='utf-8-sig')

### ↑ 여기에서 반환된 donotcall_list.csv 파일을 두낫콜에 넣고, 뱉어진 파일을 업로드한 후 파일명을 아래 filename 변수에 담기

In [26]:
filename = '2058121711_20201221094633.csv'

donotcall = pd.read_csv(filename)
donotcall['Result'] = donotcall['Result'].str.replace('\D|\s', '', regex=True)

df.loc[(df['filter'].isnull()==True) & (~df['telf2'].isin(donotcall['Result'])), 'filter'] = 14

In [27]:
# 22. 최종접촉일 3개월 이내
df.loc[(df['filter'].isnull()==True) & (df['final_dialdate'].str[0:10] >= final_call_boundary), 'filter'] = 15

In [28]:
df['filter'].fillna(0).astype(int).value_counts(dropna=False).sort_index()

0      26688
1       2880
2     673540
3       4406
4         88
5       1703
6      30498
7       2365
8       2844
9       2861
10      1927
11       916
12      1883
13      1427
14       668
15     18745
Name: filter, dtype: int64

In [29]:
gigaip.head()

,ctnum_hash,ctnum,rqnum,valid_sdate,valid_edate,ins_type,ins_amt,ins_amt_type,ins_payment,ins_stat,ins_user4,ins_stat_nm,ins_amt_type_nm,gad_start,ctnum_old,info
0,82450d9e4138aaf8ae4f1fed61bc5958ba0b8cb24030c9...,218123803,RT5722748,20191125,00000000,02,280.0,50,280.0,1,TM,None,None,20181128,None,case1
1,e75af5ced3ab864541bf99ccbb3170a85399bfd3886bd5...,220004123,RT5929483,20200108,00000000,02,280.0,50,280.0,1,,None,None,20200108,None,case1
2,af9a639595990508cce4b78a560eedd6ad99d3b9916bd9...,219153464,RT5468065,20190930,00000000,02,280.0,50,280.0,1,,None,None,20190927,None,case1
3,48105bf2f5f695b18f92ca6fae21b69e0e37227c32f5a4...,218136273,RT4574093,20190323,00000000,02,280.0,50,280.0,1,,None,None,20181224,None,case1
4,22f13a76ea900003c92b6bf4ae211bac5f15a3877fac8d...,217093589,RT4068050,20181207,00000000,02,280.0,50,280.0,1,,None,None,20170922,None,case1


In [30]:
df['기가입여부'] = np.where(df['ctnum'].isin(gigaip['ctnum']), 1, 0)
df['기가입여부'].value_counts()

0    718817
1     54622
Name: 기가입여부, dtype: int64

In [31]:
df.columns

Index(['filter', 'kunnr', 'ctnum', 'conn_date', 'ctnum_hash', 'kunnr_hash',
       'cstat', 'gad_name', 'gad_start', 'conn_start', 'conn_end',
       'monexp_amt', 'conn_type', 'ctype_l', 'ctype_m', 'ctype_s', 'itype_l',
       'itype_m', 'itype_s', 'name1', 'gad_add01', 'post_add01',
       'first_bildate', 'telf2', 'line_type', 'pck_str', 'stcd2', 'uptae',
       'jongmok', 'gros_code', 'gross_type', 'kvgr4', 'gad_pstlz', 'gad_add02',
       'post_add02', 'sms_yn', 'email_yn', 'tm_yn', 'dm_yn', 'sk1_yn',
       'sk2_yn', 'adt1_yn', 'aedat', 'aezet', 'aenam', 'nsok', 'con_cnt',
       'final_dialdate', 'resultcd', 'telf_valid_1', 'telf_valid_2',
       'telf_valid_final', '기가입여부'],
      dtype='object')

In [32]:
df.loc[(df['filter'].isnull()==True) & ((df['기가입여부'] == 1) | ((df['con_cnt']>=5)&(df['final_dialdate'].str[0:10] < final_call_boundary_2))), 'filter'] = 0

In [33]:
df.loc[(df['filter'].isnull()==True), 'filter'] = 16

In [34]:
df['filter'].fillna(0).astype(int).value_counts(dropna=False).sort_index()

0      10206
1       2880
2     673540
3       4406
4         88
5       1703
6      30498
7       2365
8       2844
9       2861
10      1927
11       916
12      1883
13      1427
14       668
15     18745
16     16482
Name: filter, dtype: int64

In [35]:
df.drop(['ctnum_hash', 'kunnr_hash', 'telf_valid_1', 'telf_valid_2', 'telf_valid_final'], axis=1, inplace=True)

In [36]:
df = df.rename(columns={
      'gubun' : '구분',
      'filter' : '필터',
      'kunnr' : '고객번호',
      'ctnum' : '계약번호',
      'cstat' : '계약상태',
      'gad_name' : '상호',
      'gad_start' : '경비개시일',
      'conn_start' : '계약개시일',
      'conn_end' : '계약종료일',
      'monexp_amt' : '월정료',
      'conn_type' : '계약타입',
      'ctype_l' : '채널(대)',
      'ctype_m' : '채널(중)',
      'ctype_s' : '채널(소)',
      'itype_l' : '업종(대)',
      'itype_m' : '업종(중)',
      'itype_s' : '업종(소)',
      'name1' : '이름1',
      'gad_add01' : '경비물건주소',
      'post_add01' : '우편물수령지주소1',
      'first_bildate' : '최초청구일',
      'telf2' : '전화번호',
      'line_type' : '관제회선구분',
      'uptae' : '업태',
      'jongmok' : '종목',
      'gros_code' : '그로스코드',
      'gross_type' : '그로스타입',
      'kvgr4' : '세그먼트',
      'gad_pstlz' : '우편번호',
      'gad_add02' : '경비물건주소상세',
      'post_add02' : '우편물수령지주소2',
      'sms_yn' : 'SMS동의',
      'email_yn' : 'EMAIL동의',
      'adt1_yn' : '개인정보수집이용동의',
      'tm_yn' : 'TM동의',
      'dm_yn' : 'DM동의',
      'sk1_yn' : 'SK1동의',
      'sk2_yn' : 'SK2동의',
      'aedat' : '작업일자',
      'aezet' : '작업시간',
      'aenam' : '변경자',
      'nsok' : 'n사계약번호',
      'final_dialdate' : '최종접촉일시',
      'resultcd' : '최종접촉결과',
      'con_cnt' : '접촉회차'
})

In [37]:
df = df[
    ['필터', '고객번호', '계약번호', '계약상태', '상호', '경비개시일', '계약개시일', '계약종료일', 
     '월정료', '계약타입', '채널(대)', '채널(중)', '채널(소)', '업종(대)', '업종(중)', '업종(소)', 
     '이름1', '경비물건주소', '우편물수령지주소1', '최초청구일', '전화번호',
     '관제회선구분', '업태', '종목', '그로스코드', '그로스타입', '세그먼트', '우편번호', '경비물건주소상세',
     '우편물수령지주소2', '개인정보수집이용동의', 'SMS동의', 'EMAIL동의', 'TM동의', 'DM동의', 'SK1동의', 'SK2동의', 
     '작업일자', '작업시간', '변경자', 'n사계약번호', '최종접촉일시', '최종접촉결과', '접촉회차', '기가입여부']]

In [38]:
workdt = datetime.datetime.now()
workdt = str(workdt.year) + str(workdt.month).zfill(2) + str(workdt.day).zfill(2)

filename = '캡스클린케어_DB_' + workdt + '.csv'
filename

'캡스클린케어_DB_20201222.csv'

In [39]:
df.to_csv(filename, encoding='utf-8-sig')

In [40]:
df['필터'].fillna(0).astype(float).value_counts(dropna=False).sort_index()

0.0      10241
1.0       2880
2.0     673540
3.0       4406
4.0         88
5.0       1703
6.0      30498
7.0       2365
8.0       2844
9.0       2850
9.5         11
10.0      1927
11.0       912
12.0      1839
13.0      1429
14.0       669
15.0     18532
16.0     16705
Name: 필터, dtype: int64

In [41]:
df[df['계약번호']=='220232603'].TM동의

702006    None
Name: TM동의, dtype: object